In [ ]:
pip install pandas numpy xgboost scikit-learn imbalanced-learn


In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score
import joblib


In [30]:
file_path = "transactions_train.csv"
df = pd.read_csv(file_path)

In [31]:
df

,transaction_amount,transaction_date,transaction_channel,is_fraud,transaction_payment_mode_anonymous,payment_gateway_bank_anonymous,payer_browser_anonymous,payer_email_anonymous,payee_ip_anonymous,payer_mobile_anonymous,transaction_id_anonymous,payee_id_anonymous
0,3606.0,2024-11-01 00:00:06,w,0,10,0,2993,ed340a2dbe10dda41f6f3d13062d039cb879008fc2106f...,773220d255be3b46b7bacc6ef9bc3174aeb7fa96208096...,NaN,ANON_0,ANON_0
1,599.0,2024-11-01 00:00:08,mobile,0,10,6,3563,6cf63ef7f4782059aa490765be32afeb1b9e87cd093bca...,773220d255be3b46b7bacc6ef9bc3174aeb7fa96208096...,XXXXX180.0,ANON_1,ANON_1
2,30.0,2024-11-01 00:00:12,w,0,10,0,2454,ed340a2dbe10dda41f6f3d13062d039cb879008fc2106f...,773220d255be3b46b7bacc6ef9bc3174aeb7fa96208096...,NaN,ANON_2,ANON_0
3,99.0,2024-11-01 00:00:34,mobile,0,11,0,1023,33946b46bf9e7d45468661551a1d810e3051b9a9fb935a...,773220d255be3b46b7bacc6ef9bc3174aeb7fa96208096...,XXXXX653.0,ANON_3,ANON_2
4,299.0,2024-11-01 00:01:23,mobile,0,11,0,3683,8aa13ceb67f69868b3ab8cf2d75b7336413e64237ba858...,773220d255be3b46b7bacc6ef9bc3174aeb7fa96208096...,XXXXX353.0,ANON_4,ANON_1
...,...,...,...,...,...,...,...,...,...,...,...,...
172921,126.0,2024-11-06 23:59:49,mobile,0,0,6,3661,4a66df51a734386b0176e03828c4425c4bb469c1936266...,7273713fb5d4f010d68e1ff0443134eb215070a47b93d5...,XXXXX155.0,ANON_172921,ANON_10
172922,100.0,2024-11-06 23:59:49,w,0,10,0,1605,ed340a2dbe10dda41f6f3d13062d039cb879008fc2106f...,7273713fb5d4f010d68e1ff0443134eb215070a47b93d5...,NaN,ANON_172922,ANON_0
172923,299.0,2024-11-06 23:59:52,mobile,0,11,0,3721,8f9cb7af1e4494eb3d290f8807328dac5c3352177583b1...,7273713fb5d4f010d68e1ff0443134eb215070a47b93d5...,XXXXX609.0,ANON_172923,ANON_1
172924,200.0,2024-11-06 23:59:54,w,0,10,0,3183,ed340a2dbe10dda41f6f3d13062d039cb879008fc2106f...,7273713fb5d4f010d68e1ff0443134eb215070a47b93d5...,NaN,ANON_172924,ANON_0


In [32]:
imputer = SimpleImputer(strategy='median')
df['transaction_amount'] = imputer.fit_transform(df[['transaction_amount']])

In [33]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df['transaction_date'] = df['transaction_date'].apply(lambda x: np.random.choice(df['transaction_date'].dropna()) if pd.isnull(x) else x)


In [34]:
df['transaction_date'] = df['transaction_date'].astype(int) / 10**9  # Convert to seconds since epoch

In [35]:
df = df.drop(columns=['payer_mobile_anonymous'])

In [36]:
encoder = LabelEncoder()
categorical_columns = ['transaction_channel', 'payer_email_anonymous', 'payee_ip_anonymous']
for col in categorical_columns:
    df[col] = encoder.fit_transform(df[col].astype(str))

In [37]:
X = df.drop(columns=['is_fraud', 'transaction_id_anonymous', 'payee_id_anonymous'])
y = df['is_fraud']

In [38]:

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [40]:

model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:14:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [41]:
joblib.dump(model, "fraud_detection_xgboost.pkl")

['fraud_detection_xgboost.pkl']

In [42]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9998843362345661
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     34686
           1       1.00      1.00      1.00     34480

    accuracy                           1.00     69166
   macro avg       1.00      1.00      1.00     69166
weighted avg       1.00      1.00      1.00     69166



In [43]:
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", np.mean(cv_scores))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:14:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:14:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:15:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:15:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:15:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

Cross-validation scores: [0.99976506 0.99987349 0.99983735 0.99985542 0.99985542]
Mean CV accuracy: 0.9998373478951736


In [44]:
df_copy = df.copy()

encoder = LabelEncoder()
categorical_columns = ['transaction_channel', 'payer_email_anonymous', 'payee_ip_anonymous', 'transaction_id_anonymous', 'payee_id_anonymous']
for col in categorical_columns:
    df_copy[col] = encoder.fit_transform(df_copy[col].astype(str))

In [45]:
X = df_copy.drop(columns=['is_fraud'])
y = df_copy['is_fraud']

In [46]:
correlations = X.corrwith(y).abs().sort_values(ascending=False)
print("Feature correlations with target:")
print(correlations)

Feature correlations with target:
payer_browser_anonymous               0.013583
payer_email_anonymous                 0.009399
transaction_payment_mode_anonymous    0.008443
transaction_channel                   0.007297
payee_id_anonymous                    0.005253
payment_gateway_bank_anonymous        0.003853
payee_ip_anonymous                    0.001701
transaction_amount                    0.001268
transaction_date                      0.000418
transaction_id_anonymous              0.000106
dtype: float64
